## Detecting-patterns-in-purchase-history-using-association-rule-learning-methods

### 1. Experimental Dataset as proof of concept

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [2]:
data_simple_input =[\
["F", "G", "H", "I", "J", "K", "M"],
["F", "H", "I", "J", "K", "L", "M"],
["F", "H", "I", "N"],
["F", "G", "J", "L", "M", "N", "R"],
["F", "G", "J", "N", "R"],
["F", "G", "M", "N", "R"],
["F", "K", "N"],
["F", "G", "I", "R"],
["G", "H", "N"],
["G", "J", "R"]]

# Counts:
# F:8
# G:7
# N:6
# J:5
# R:5
# H:4
# I:4
# M:4
# K:3


### 1. Traditional alorthms from libraries

#### 1.1 Apriori
APriori is the first tradtional ruiles algorithm.

In [3]:
# Imports
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

In [4]:
te = TransactionEncoder()
te_ary = te.fit(data_simple_input).transform(data_simple_input)
data_simple = pd.DataFrame(te_ary, columns=te.columns_)
data_simple

,F,G,H,I,J,K,L,M,N,R
0,True,True,True,True,True,True,False,True,False,False
1,True,False,True,True,True,True,True,True,False,False
2,True,False,True,True,False,False,False,False,True,False
3,True,True,False,False,True,False,True,True,True,True
4,True,True,False,False,True,False,False,False,True,True
5,True,True,False,False,False,False,False,True,True,True
6,True,False,False,False,False,True,False,False,True,False
7,True,True,False,True,False,False,False,False,False,True
8,False,True,True,False,False,False,False,False,True,False
9,False,True,False,False,True,False,False,False,False,True


In [5]:
from mlxtend.frequent_patterns import apriori

apriori(data_simple, min_support=0.5,use_colnames=True)

,support,itemsets
0,0.8,(F)
1,0.7,(G)
2,0.5,(J)
3,0.6,(N)
4,0.5,(R)
5,0.5,"(F, G)"
6,0.5,"(N, F)"
7,0.5,"(R, G)"


#### 1.2 FP-Growth
FP-Growth gives exactly the same output, but is much more efficient, especially when using big data.

In [6]:
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(data_simple, min_support=0.5,use_colnames=True)

,support,itemsets
0,0.8,(F)
1,0.7,(G)
2,0.5,(J)
3,0.6,(N)
4,0.5,(R)
5,0.5,"(F, G)"
6,0.5,"(F, N)"
7,0.5,"(R, G)"


#### 2. Modified implementation of FP-Growth as Basis

#### 2.1 Classical implementation for proving correct algorithm

The modified implementation of FP-Growth will give back the same result as the classical algorithms.
This means, without any date & profit parameter for weighted support, basically thrmalgorithm works as expected.
One fundamental modification has been done, when finding the antecedent leading to the the combined new antecedent & consequent.
The modification only considers "one antecedent", instead of many. The "one antecedent" is taken from the strict tree-structure (path) from high support to low support.
##### Advantages:
- Clearer rules & No duplication of changing antecedent leading to the  same itemset
- Much better performance, especially for big data. All figures can be derived from one Loop through all transactions O(n)-complexity for whole dataset. Inside the paths there is O(n*2) complexity, but the paths are usually very small compared to the whole dataset. Alternatively all combinations of paths have to be looped again
##### Disadvantages:
- Potential loss of information, if onbe rule for a very special antecedent leading to the itemset was decisive

In [7]:
data_simple = pd.read_csv("datasets/proof_of_concept/transactions.csv")
df_data_simple = data_simple.groupby("transaction",dropna=True)["item"].agg([lambda x: list(x),"count"])
df_data_simple

,<lambda_0>,count
transaction,,
T01,"[F, G, H, I, J, K, M]",7
T02,"[F, H, I, J, K, L, M]",7
T03,"[F, H, I, N]",4
T04,"[F, G, J, L, M, N, R]",7
T05,"[F, G, J, N, R]",5
T06,"[F, G, M, N, R]",5
T07,"[F, K, N]",3
T08,"[F, G, I, R]",4
T09,"[G, H, N]",3


In [8]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

rules = mod_fp_growth.fpgrowthFromDataFrame(df_data_simple, minSupRatio=0.5, maxSupRatio=1, minConf=0, item_col=1) #Traditional Association Rules

rules

,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement
4,[],NA,[F],8,[F],8,0.8,NA,NA,NA
3,[],NA,[G],7,[G],7,0.7,NA,NA,NA
2,[],NA,[N],6,[N],6,0.6,NA,NA,NA
0,[],NA,[J],5,[J],5,0.5,NA,NA,NA
1,[],NA,[R],5,[R],5,0.5,NA,NA,NA


#### 2.2 Weighted support with date-decay function

The idea of a date support decay function is, that recent transactions should have normally more weight, than older ones.
There are used the following paramters:
x will be determined between [0,1]
- max_date=datetime.datetime(2022, 11, 10),
-> This is the max date, x of max_date is 1
- date_range=10,
-> This is the range of x, max_date - range = 0
- date_sensitivity = lambda x: 1 / (1 + math.exp(-10*x+5))
-> This is the function used for date exemplatory. It is a modfied sigmoid, using the curve range [0,1] to represent date decay
-> This has still to be calibrated and could differ for every new Dataset. For example the curve could be rather flat around 1 for only a small effect
-> In the example with lambda x: 1 / (1 + math.exp(-10*x+5)), the curve is quite extreme and maybe overvalues recent events

In [9]:
data_simple = pd.read_csv("datasets/proof_of_concept/transactions.csv")
data_simple["date"] = pd.to_datetime(data_simple["date"],format='%Y-%m-%d')
df_data_simple_withdate = data_simple.groupby("transaction",dropna=True)["item","date"].agg([lambda x: list(x)])
df_data_simple_withdate

C:\Users\david\AppData\Local\Temp/ipykernel_24036/64192650.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_data_simple_withdate = data_simple.groupby("transaction",dropna=True)["item","date"].agg([lambda x: list(x)])


,item,date
,<lambda>,<lambda>
transaction,,
T01,"[F, G, H, I, J, K, M]","[2022-11-01 00:00:00, 2022-11-01 00:00:00, 202..."
T02,"[F, H, I, J, K, L, M]","[2022-11-02 00:00:00, 2022-11-02 00:00:00, 202..."
T03,"[F, H, I, N]","[2022-11-02 00:00:00, 2022-11-03 00:00:00, 202..."
T04,"[F, G, J, L, M, N, R]","[2022-11-04 00:00:00, 2022-11-04 00:00:00, 202..."
T05,"[F, G, J, N, R]","[2022-11-05 00:00:00, 2022-11-05 00:00:00, 202..."
T06,"[F, G, M, N, R]","[2022-11-06 00:00:00, 2022-11-06 00:00:00, 202..."
T07,"[F, K, N]","[2022-11-07 00:00:00, 2022-11-07 00:00:00, 202..."
T08,"[F, G, I, R]","[2022-11-08 00:00:00, 2022-11-08 00:00:00, 202..."


In [10]:
import math
import datetime

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    df_data_simple_withdate,
    minSupRatio=0.3,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    date_col=2,
    max_date=datetime.datetime(2022, 11, 10),
    date_range=10,
    date_sensitivity = lambda x: 1 / (1 + math.exp(-10*x+5))
    ) #Only Date


rules

,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement
4,[],NA,[G],4.445881,[G],4.445881,0.820042,NA,NA,NA
3,[],NA,[N],3.482014,[N],3.482014,0.642257,NA,NA,NA
2,[],NA,[F],3.446209,[F],3.446209,0.635652,NA,NA,NA
1,[],NA,[R],3.445881,[R],3.445881,0.635592,NA,NA,NA
0,[],NA,[J],1.827661,[J],1.827661,0.337112,NA,NA,NA


#### 2.3 Weighted support with profit-dependent function

The main driver for business it not the frequency of items, but the proftibility. One can normally assume, that more frequent items are lower in price, higher in margin, but can be equal with not frequent but highly priced articles.
I've reas in one article, one reason association rules are not applied that often is the lack of relevance. Frequency is only one part, but not the ultimate driver for business.
Instead of just counting each transaction, we weight each article of every transation and set it in a relationship with association rules.
The result has to be interpreted carefully. It cannot be interpreted the same as the traditional methods. The sup_ant&cons is just the profit resulting from a relation. The % of sup_ant&cons represents the percentage of the whole profit of sum of all articles.

The interference of profit and frequency could be solved post association rules creation by business when later connecting all frequencies with profit from articles. However, there we have the problem, that we sorted out the least frequent articles already because of performance or releance and lose crucial relations. Moreover this approach would be not that straightforward.


In [11]:
data_simple = pd.read_csv("datasets/proof_of_concept/transactions.csv")
df_data_simple_withprofit = data_simple.groupby("transaction",dropna=True)["item","profit"].agg([lambda x: list(x)])
df_data_simple_withprofit

C:\Users\david\AppData\Local\Temp/ipykernel_24036/4188091503.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_data_simple_withprofit = data_simple.groupby("transaction",dropna=True)["item","profit"].agg([lambda x: list(x)])


,item,profit
,<lambda>,<lambda>
transaction,,
T01,"[F, G, H, I, J, K, M]","[10, 20, 30, 40, 50, 60, 80]"
T02,"[F, H, I, J, K, L, M]","[10, 30, 40, 50, 60, 70, 80]"
T03,"[F, H, I, N]","[10, 30, 40, 90]"
T04,"[F, G, J, L, M, N, R]","[10, 20, 50, 70, 80, 90, 100]"
T05,"[F, G, J, N, R]","[10, 20, 50, 90, 100]"
T06,"[F, G, M, N, R]","[10, 20, 80, 90, 100]"
T07,"[F, K, N]","[10, 60, 90]"
T08,"[F, G, I, R]","[10, 20, 40, 100]"


In [13]:
import math
import datetime

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    df_data_simple_withprofit,
    minSupRatio=0.03,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    profit_col=2,
    max_profit = 100,
    profit_sensitivity = lambda x : 1 * x
    ) #Only Date

rules


,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement,profit_associated,perc_of_total_profit,profit_associated_prev,net_change,profit_last_item,loss_by_change
296,"[F, N, G]",3,[R],5,"[R, N, F, G]",3,0.3,1.0,2.0,0.5,660.0,0.271605,360.0,300.0,300.0,0.0
293,"[N, G]",4,[R],5,"[R, N, G]",3,0.3,0.75,1.5,0.25,630.0,0.259259,440.0,190.0,300.0,-110.0
291,"[F, N]",5,[R],5,"[R, N, F]",3,0.3,0.6,1.2,0.1,600.0,0.246914,500.0,100.0,300.0,-200.0
272,"[R, N, F, G]",3,[M],4,"[R, F, G, N, M]",2,0.2,0.666667,1.666667,0.266667,600.0,0.246914,660.0,-60.0,160.0,-220.0
287,[G],7,[R],5,"[R, G]",5,0.5,0.714286,1.428571,0.214286,600.0,0.246914,140.0,460.0,500.0,-40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,"[H, G]",2,[I],4,"[H, I, G]",1,0.1,0.5,1.25,0.1,90.0,0.037037,100.0,-10.0,40.0,-50.0
2,[G],7,[L],2,"[L, G]",1,0.1,0.142857,0.714286,-0.057143,90.0,0.037037,140.0,-50.0,70.0,-120.0
122,[G],7,[K],3,"[K, G]",1,0.1,0.142857,0.47619,-0.157143,80.0,0.032922,140.0,-60.0,60.0,-120.0
307,[],NA,[F],8,[F],8,0.8,NA,NA,NA,80.0,0.032922,0.0,0.0,80.0,0.0


#### 2.4 Combined weighted suppport of date-decay and profit dependent function

#### 3. Analysis of Kaggle Dataset

#### 3.1 Investigating Dataset

#### 3.1 Description
Kaggle Dataset: https://www.kaggle.com/datasets/mkechinov/ecommerce-purchase-history-from-electronics-store <br>
The Dataset is Open-Source <br>
This Dataset contains purchase data from April 2020 to November 2020 from a large home appliances and electronics online store. <br>
Each row in the file represents an event. All events are related to products and users. Each event is like many-to-many relation between products and users. <br>

In [14]:
data = pd.read_csv("datasets/ecommerce_purchase_history_from_electronic_store/dataset_part_1.csv") #Smaller Subset of Original Kaggle DataSet for Testing

data

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18
...,...,...,...,...,...,...,...,...
799993,2020-03-12 10:21:36 UTC,2348817879496917363,1515966223509298265,2.268105e+18,NaN,rockstar,60.16,NaN
799994,2020-03-12 10:21:36 UTC,2348817879496917363,2273948241848435411,2.268105e+18,NaN,none,29.14,NaN
799995,2020-03-09 12:51:59 UTC,2348817879522083188,1515966223509250877,2.268105e+18,computers.notebook,ipower,2.29,NaN
799996,2020-03-09 12:51:59 UTC,2348817879522083188,1515966223509088654,2.268105e+18,NaN,rowenta,34.70,NaN


In [15]:
data["date"] = pd.to_datetime(data['event_time']).dt.date
max_date = data["date"].max()
data["margin"] = 0.1
data["profit"] = data["price"] * data["margin"]
max_profit = data["profit"].max()

data = data.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))

data

C:\Users\david\AppData\Local\Temp/ipykernel_24036/1773867838.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data = data.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))


,product_id,date,price,margin
order_id,,,,
2294359932054536986,"[1515966223509089906, 1515966223509089906]","[2020-04-24, 2020-04-24]","[162.01, 162.01]","[0.1, 0.1]"
2294444024058086220,"[2273948319057183658, 2273948319057183658]","[2020-04-24, 2020-04-24]","[77.52, 77.52]","[0.1, 0.1]"
2294584263154074236,[2273948316817424439],[2020-04-24],[217.57],[0.1]
2295716521449619559,[1515966223509261697],[2020-04-26],[39.33],[0.1]
2295740594749702229,"[1515966223509104892, 1515966223509104892, 151...","[2020-04-26, 2020-04-26, 2020-04-26, 2020-04-26]","[1387.01, 1387.01, 1387.01, 1387.01]","[0.1, 0.1, 0.1, 0.1]"
...,...,...,...,...
2348817879136207217,[1515966223509261668],[2020-03-13],[4.14],[0.1]
2348817879480140146,[1515966223509341137],[2020-03-13],[12.25],[0.1]
2348817879496917363,"[1515966223509107816, 1515966223509127863, 151...","[2020-03-12, 2020-03-12, 2020-03-12, 2020-03-1...","[59.24, 57.85, nan, 41.64, 543.96, 481.46, 60....","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"


#### 3.2 Dataset Part 1 with profit

In [18]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

#For better trackingI take price instead of profit as profit, because tracking ius easier and at moment I anyway just multiply by 0.1 for profit
# We see how "super efficient it is, only 6.8 seconds for 13k rules inclduing writing to Excel"
rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.001,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    profit_col=3,
    max_profit = max_profit,
    profit_sensitivity = lambda x : 1 * x
    )

rules.to_excel("fp_groth_out.xlsx",index=False) 
rules


,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement,profit_associated,perc_of_total_profit,profit_associated_prev,net_change,profit_last_item,loss_by_change
13090,[],NA,[2273948218662322995],4422,[2273948218662322995],4422,0.011145,NA,NA,NA,816831.840000,0.009808,0.000000,0.000000,816831.84,0.000000
10194,[],NA,[1515966223509088567],785,[1515966223509088567],785,0.001978,NA,NA,NA,672140.550000,0.008070,0.000000,0.000000,672140.55,0.000000
13059,[2273948218662322995],4422,[2273948218662322996],2927,"[2273948218662322996, 2273948218662322995]",1983,0.004998,0.44844,60.790457,0.441063,651812.100000,0.007826,816831.840000,-165019.740000,285512.34,-450532.080000
10586,[],NA,[1515966223509089533],833,[1515966223509089533],833,0.002099,NA,NA,NA,636103.790000,0.007638,0.000000,0.000000,636103.79,0.000000
13082,[],NA,[2273948316473492113],3852,[2273948316473492113],3852,0.009708,NA,NA,NA,624224.530000,0.007495,0.000000,0.000000,624224.53,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11593,"[2273948186248741817, 2273948316473492113]",96,[2273948312304353950],1090,"[2273948186248741817, 2273948312304353950, 227...",1,0.000003,0.010417,3.791896,0.00767,397.882059,0.000005,24888.197632,-24490.315574,138.63,-24628.945574
10309,"[2273948186248741817, 1515966223509089598]",38,[2273948312304353946],801,"[2273948186248741817, 2273948312304353946, 151...",1,0.000003,0.026316,13.035811,0.024297,397.840000,0.000005,8961.540000,-8563.700000,162.01,-8725.710000
12010,"[2273948186248741817, 1515966223509089598]",38,[1515966223509089076],1191,"[2273948186248741817, 1515966223509089076, 151...",1,0.000003,0.026316,8.767157,0.023314,397.610000,0.000005,8961.540000,-8563.930000,161.78,-8725.710000
10812,"[2273948186248741817, 1515966223509104162]",26,[1515966223509089448],877,"[2273948186248741817, 1515966223509089448, 151...",1,0.000003,0.038462,17.401281,0.036251,397.610000,0.000005,6733.480000,-6335.870000,138.63,-6474.500000


#### 3.3 Sensitivity Analysis

In [ ]:
import math
import datetime

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    df_data_simple_withprofit,
    minSupRatio=0.03,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    profit_col=2,
    max_profit = 100,
    profit_sensitivity = lambda x : 1 * x
    ) #Only Date

rules
